In [ ]:
import re
from functools import cache
from collections import defaultdict

In [ ]:
def parse_data(my_file) -> list:
    with open(my_file) as f:
        return [line.strip() for line in f.readlines()]

In [ ]:
def coords(keys: list) -> dict:
    result = {}
    for y, line in enumerate(keys):
        for x, ch in enumerate(line):
            result[ch] = y + x * 1j
    del result[' ']
    return result

In [ ]:
@cache
def level(code: str, is_dir: bool = True) -> list:
    global num_keys
    global dir_keys
    keypad = dir_keys if is_dir else num_keys
    route = []
    current = 'A'
    for new in code:
        new_route = keypad[new] - keypad[current]
        horisontal = abs(int(new_route.imag))*('<' if new_route.imag < 0 else '>')
        vertical = abs(int(new_route.real)) * ('^' if new_route.real < 0 else 'v')
        if not is_dir and ((new_route.imag < 0 and not (new in ('1','4','7') and current in ('0','A'))) or (new in ('0','A') and current in ('1','4','7'))):
            route.append(horisontal+vertical+'A')
        elif is_dir and (current == '<' or (new in ['^','v'] and current in ['>','A'])):
            route.append(horisontal+vertical+'A')
        else:
            route.append(vertical+horisontal+'A')
        current = new
    return route

In [ ]:
def complexities(data: list, keypads:int = 2) -> int:
    result = 0
    for code in data:
        current = defaultdict(int)
        for item in level(code, False):
            current[item] += 1
        for _ in range(keypads):
            new = defaultdict(int)
            for k,v in current.items():
                for l in level(k):
                    new[l]+=v
            current = new
        result += int(code[:-1])*sum(len(k)*v for k,v in current.items())
    return result

In [ ]:
%%time
data = parse_data('raw.txt')
num_keys = coords(['789','456','123',' 0A'])
dir_keys = coords([' ^A', '<v>'])
print('Part 1: ', complexities(data))
print('Part 2: ', complexities(data,25))

In [ ]:
#tests
test1 = parse_data('test_raw.txt')
assert complexities(test1) == 126384